In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
dataPath = "/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/data/task44"
fileList = os.listdir(dataPath)
filePathList = [dataPath+"/"+fileName for fileName in fileList]

In [16]:
dataFilterList = []
columnsList = ['version','ts','seg','deviceId','fc','pk','ak','logId','messageId','content','dir','dataana','extended']
for filePath in filePathList:
    dataDfList = pd.read_csv(filePath, compression='zip', chunksize=10000, usecols=columnsList)
    for dataDf in dataDfList:
        notNaDf = dataDf[dataDf["dataana"].notna()]
        if (len(notNaDf) == 0):continue
        f1Df = notNaDf[(notNaDf["dataana"].str.contains('sequence')) & (notNaDf['seg'] == "IOT2ASR")]
        f2Df = f1Df[f1Df["dataana"].str.contains("轻柔|标准擦|强力擦|标准|强力|Max")]
        dataFilterList += f2Df.values.tolist()
resultData = pd.DataFrame(dataFilterList, columns=columnsList)
resultData.head()

,version,ts,seg,deviceId,fc,pk,ak,logId,messageId,content,dir,dataana,extended
0,2.1,2024-07-04 21:12:20.247,IOT2ASR,ze0mpx229x9z0zP000_000213078,ze0mpx,229x9z0z,P000_000213078,9EE7AF73-58CF-4498-BC18-3BD99EA36960,17200987279402,"{""bdvs-version"": ""2.0.0"", ""directives"": [{""hea...",ResponseNluHandle,"{""payload"": {""extension"": ""{\""origin\"":{\""quer...",NaN
1,2.1,2024-07-04 21:00:28.782,IOT2ASR,ze0mpx229x9z0zP000_000263778,ze0mpx,229x9z0z,P000_000263778,5E945CB7-DE69-4F8E-8489-09B62C3B6106,17200980184701,"{""bdvs-version"": ""2.0.0"", ""directives"": [{""hea...",ResponseNluHandle,"{""payload"": {""extension"": ""{\""origin\"":{\""quer...",NaN
2,2.1,2024-07-04 21:17:51.176,IOT2ASR,ze0mpx229x9z0zP000_000232777,ze0mpx,229x9z0z,P000_000232777,06D7C0F6-3485-4632-A75A-44DF625B3443,17200990639580,"{""bdvs-version"": ""2.0.0"", ""directives"": [{""hea...",ResponseNluHandle,"{""payload"": {""extension"": ""{\""origin\"":{\""quer...",NaN
3,2.1,2024-07-04 21:17:40.511,IOT2ASR,ze0mpx229x9z0zP000_000232777,ze0mpx,229x9z0z,P000_000232777,67FDAD4B-DEAC-4019-A475-599BF5A6A286,17200990528097,"{""bdvs-version"": ""2.0.0"", ""directives"": [{""hea...",ResponseNluHandle,"{""payload"": {""extension"": ""{\""origin\"":{\""quer...",NaN
4,2.1,2024-07-04 21:27:19.031,IOT2ASR,ze0mpx229x9z0zP000_000036535,ze0mpx,229x9z0z,P000_000036535,B930913F-7584-45DE-AEB4-AF33C5AFB94C,17200996260471,"{""bdvs-version"": ""2.0.0"", ""directives"": [{""hea...",ResponseNluHandle,"{""payload"": {""extension"": ""{\""origin\"":{\""quer...",NaN


In [31]:
import json
def extract_query_intent(dataana):
    dataanaJson = json.loads(dataana)
    intents = []
    extension = dataanaJson.get("payload", {}).get("extension", {})
    if isinstance(extension, str):
        extension = json.loads(extension)
    results = extension.get("origin",{}).get("results",[])
    for res in results:
        intents.append(res.get("intent", None))
    query = dataanaJson.get("query", None)
    return query, intents
resultData["query"], resultData["intent"] =  zip(*resultData["dataana"].apply(extract_query_intent))
    

In [32]:
resultData[["query", "intent"]].to_csv("query_and_intent_data.csv", index=False)

In [27]:
extension["origin"]

{'query': '强力扫拖',
 'encoding': 'utf8',
 'results': [{'sequence': 1,
   'domain': 'roborock_parameter_control',
   'intent': 'set_air_suction',
   'slots': {'gear': [{'slot_type': 'STRING', 'text': '强力', 'value': '103'}]}},
  {'sequence': 2,
   'domain': 'roborock_clean_function',
   'intent': 'start_sweep_and_mop',
   'slots': {}}]}